In [1]:
import openfe
from openfe_skunkworks.protocols import openmm_afe

In [2]:
# first load chemistry components
solvent = openfe.SolventComponent(smiles='O')
benzene = openfe.SmallMoleculeComponent.from_sdf_file('./benzene.sdf')

# arrange into ChemicalSystems that represent the free energy difference
cs_solvent = openfe.ChemicalSystem({'solvent': solvent}, name='solvent system')
cs_benzene = openfe.ChemicalSystem({'solvent': solvent, 'ligand': benzene}, name='solvated benzene')

In [3]:
# grab default Settings for the AFE solvation protocol
settings = openmm_afe.AbsoluteSolvationProtocol.default_settings()

In [4]:
# in gromacs currency, this is a little like the .mdp file
settings

{'alchemical_settings': {},
 'integrator_settings': {'barostat_frequency': <Quantity(25, 'timestep')>,
                         'constraint_tolerance': 1e-06,
                         'langevin_collision_rate': <Quantity(1.0, '1 / picosecond')>,
                         'n_restart_attempts': 20,
                         'reassign_velocities': False,
                         'remove_com': False,
                         'timestep': <Quantity(4, 'femtosecond')>},
 'lambda_settings': {'lambda_elec': [0.0,
                                     0.25,
                                     0.5,
                                     0.75,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                            

In [5]:
# notably, this settings section is an OpenMMSolvationSettings
print(type(settings.solvation_settings))

<class 'openfe.protocols.openmm_utils.omm_settings.OpenMMSolvationSettings'>


In [6]:
from openfe_skunkworks.protocols.openmm_utils.omm_settings import PackmolSolvationSettings

packmol_settings = PackmolSolvationSettings()

In [7]:
type(settings)

openfe.protocols.openmm_afe.equil_afe_settings.AbsoluteSolvationSettings

In [8]:
# this currently fails, we'll either have to:
# put a solvation_settings: Union[OpenMMSolvationSettings, OpenMMSolvationSettings]
# OR create an empty base class that both the above classes inherit from, not sure which is cleaner w/ pydantic
settings.solvation_settings = packmol_settings

# we can then replace the "if interchange:" line with "if isinstance(solvation_settings, PackmolSolvationSettings):"

In [9]:
# crystallise the settings into a Protocol, a chemistry-less definition of a algorithm
protocol = openmm_afe.AbsoluteSolvationProtocol(settings)

In [10]:
# attach the chemistry by bundling the two together into a Transformation
transformation = openfe.Transformation(
    stateA=cs_benzene,
    stateB=cs_solvent,
    mapping=None,
    protocol=protocol
)

In [11]:
transformation

Transformation(stateA=ChemicalSystem(name=solvated benzene, components={'solvent': SolventComponent(name=O, Na+, Cl-), 'ligand': SmallMoleculeComponent(name=benzene)}), stateB=ChemicalSystem(name=solvent system, components={'solvent': SolventComponent(name=O, Na+, Cl-)}), protocol=<AbsoluteSolvationProtocol-08b4951f9786bdc77ff4c1d9e9f63c90>)

In [12]:
# this is a bit like a gromacs .tpr, it's a self contained simulation definition ready to go
transformation.dump('benzene.json')

In [13]:
# running "openfe quickrun benzene.json" should start the simulation
!openfe quickrun benzene.json

Loading file...
Planning simulations for this edge...
Starting the simulations for this edge...
SYSTEM CONFIG DETAILS:
INFO:openfe.utils.system_probe.log:SYSTEM CONFIG DETAILS:
hostname: 'Matts-MacBook-Pro.local'
INFO:openfe.utils.system_probe.log.hostname:hostname: 'Matts-MacBook-Pro.local'
CUDA-based GPU not found
INFO:openfe.utils.system_probe.log.gpu:CUDA-based GPU not found
Memory used: 1.7G (95.4%)
INFO:openfe.utils.system_probe.log:Memory used: 1.7G (95.4%)
/Users/mattthompson/software/openfe_skunkworks/devscripts/scratch_AbsoluteSolvationVacuumUnit-06eb09e7d98b408dacf1201bcdf01937_attempt_0: 96% full (17.7G free)
INFO:openfe.utils.system_probe.log:/Users/mattthompson/software/openfe_skunkworks/devscripts/scratch_AbsoluteSolvationVacuumUnit-06eb09e7d98b408dacf1201bcdf01937_attempt_0: 96% full (17.7G free)
setting up alchemical system
INFO:gufekey.openfe_skunkworks.protocols.openmm_afe.equil_solvation_afe_method.AbsoluteSolvationVacuumUnit:setting up alchemical system
Parameteriz